<a href="https://colab.research.google.com/github/Bipin-6751/Machine-Learning-Collection/blob/master/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [3]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


In [4]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


In [5]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


converting dollar into indian rupees

In [6]:
tf_dataset = tf_dataset.map(lambda x: x*72)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


In [7]:
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1584
1512
2304
2448
2232
2232


In [8]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[1512 2232]
[2304 1584]
[2448 2232]


In [9]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1512 2232]
[2304 2448]
[2232 1584]


In [ ]:
!unzip images.zip

In [11]:
images_ds = tf.data.Dataset.list_files('images/*/*', shuffle=False)

In [13]:
for file in images_ds.take(3):
  print(file.numpy())

b'images/cat/20 Reasons Why Cats Make the Best Pets....jpg'
b'images/cat/7 Foods Your Cat Can_t Eat.jpg'
b'images/cat/A cat appears to have caught the....jpg'


In [14]:
image_count = len(images_ds)
image_count

130

In [15]:
type(images_ds)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [16]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'images/cat/China_s First Cloned Kitten_ Garlic....png'
b'images/dog/The 20 Best Dog Breeds for Runners....jpg'
b'images/dog/Dog Breeds Banned By Home Insurance....jpg'


In [17]:
class_names = ["cat","dog"]

In [18]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [19]:
len(train_ds)

104

In [20]:
len(test_ds)

26

In [21]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [23]:
get_label("images/dog/20 Reasons Why Cats Make the Best Pets....jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'dog'>

In [24]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label

In [25]:
img, label = process_image("images/cat/20 Reasons Why Cats Make the Best Pets....jpg")
img.numpy()[:2]

array([[[155.      , 186.      , 215.      ],
        [156.      , 187.      , 216.      ],
        [158.      , 189.      , 218.      ],
        [160.0039  , 189.0039  , 219.0039  ],
        [161.0039  , 190.0039  , 220.0039  ],
        [162.      , 191.      , 221.      ],
        [166.      , 193.      , 222.      ],
        [167.      , 194.      , 223.      ],
        [168.      , 195.      , 224.      ],
        [169.      , 196.      , 225.      ],
        [170.      , 197.      , 224.      ],
        [170.0039  , 197.0039  , 224.0039  ],
        [172.      , 199.      , 226.      ],
        [173.0039  , 199.0039  , 224.0039  ],
        [174.0039  , 200.0039  , 225.0039  ],
        [175.0039  , 201.0039  , 226.0039  ],
        [176.      , 202.      , 227.      ],
        [177.0039  , 203.0039  , 228.0039  ],
        [177.0039  , 203.0039  , 228.0039  ],
        [179.0039  , 203.0039  , 227.0039  ],
        [180.0039  , 204.0039  , 228.0039  ],
        [180.0039  , 204.0039  , 2

In [26]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [27]:
for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[224.       231.       241.      ]
  [226.       233.       243.      ]
  [228.       230.       242.      ]
  ...
  [220.5      227.5      233.5     ]
  [221.94922  228.94922  238.94922 ]
  [197.37988  204.37988  214.37988 ]]

 [[228.       231.       238.      ]
  [228.       231.       238.      ]
  [228.75     230.75     242.75    ]
  ...
  [224.       231.       241.      ]
  [222.       229.       239.      ]
  [222.       229.       239.      ]]

 [[228.       231.       238.      ]
  [225.94922  228.94922  235.94922 ]
  [228.31348  230.62695  241.68652 ]
  ...
  [224.       231.       240.37305 ]
  [222.       229.       238.92383 ]
  [222.       229.       238.52539 ]]

 ...

 [[214.46484  146.5752   117.4707  ]
  [165.0039    62.376953  32.063477]
  [239.51172  188.39941  158.88086 ]
  ...
  [207.48535  134.28027  109.88086 ]
  [194.38086  113.34668   86.93164 ]
  [234.48438  191.06055  166.62695 ]]

 [[181.2334    80.9707    56.442383]
  [214.22461  157.166

In [28]:
def scale(image, label):
    return image/255, label

In [29]:
train_ds = train_ds.map(scale)

In [30]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.15179132 0.06159524 0.00154622]
****Label:  b'cat'
****Image:  [0.8056679  0.7507659  0.74684435]
****Label:  b'dog'
****Image:  [0.51690507 0.42278743 0.38357174]
****Label:  b'dog'
****Image:  [0.82817096 0.8203278  0.7693474 ]
****Label:  b'cat'
****Image:  [0.6878983  0.69181985 0.672212  ]
****Label:  b'cat'
